# Notebook 7: HBOS (Histogram-based Outlier Score) - Velocidade!

## Objetivo
Usar o HBOS para detectar anomalias em datasets massivos com velocidade quase linear.

## O Conceito
HBOS é um método estatístico. Ele assume independência entre as features (o que é uma simplificação forte, mas rápida).
1.  Cria histogramas para cada feature individualmente.
2.  Calcula a probabilidade (densidade) de cada valor em seu histograma.
3.  Multiplica (ou soma os logs) das probabilidades de todas as features.
4.  Score final = Inverso da probabilidade conjunta. (Pouco provável = Anomalia).

In [ ]:
!pip install -q pyod pandas matplotlib seaborn

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from pyod.models.hbos import HBOS
from pyod.models.knn import KNN
from pyod.utils.data import generate_data

plt.rcParams['figure.figsize'] = (10, 6)

## 1. Simulação de Alta Performance
Vamos gerar um dataset grande para comparar a velocidade do HBOS vs KNN.

In [ ]:
n_train = 50000  # 50 mil amostras
n_features = 10
contamination = 0.05

X_train, _, _, _ = generate_data(n_train=n_train, n_features=n_features, contamination=contamination, random_state=42)

print(f"Dataset gerado com {n_train} linhas e {n_features} colunas.")

## 2. Comparativo de Tempo: HBOS vs KNN
O KNN precisa calcular distâncias entre todos os pontos (complexidade alta). O HBOS só precisa passar uma vez pelos dados para montar os histogramas.

In [ ]:
# Testando HBOS
start = time.time()
clf_hbos = HBOS(n_bins=20, contamination=contamination)
clf_hbos.fit(X_train)
end = time.time()
time_hbos = end - start
print(f"Tempo de Treino HBOS: {time_hbos:.4f} segundos")

# Testando KNN
start = time.time()
clf_knn = KNN(n_neighbors=5, contamination=contamination)
clf_knn.fit(X_train)
end = time.time()
time_knn = end - start
print(f"Tempo de Treino KNN: {time_knn:.4f} segundos")

print(f"\nHBOS foi {time_knn / time_hbos:.1f} vezes mais rápido!")

## 3. Visualizando os Histogramas (features individuais)
Vamos ver como o HBOS enxerga uma das features.

In [ ]:
# Pegando a feature 0
feature_0 = X_train[:, 0]

plt.hist(feature_0, bins=20, edgecolor='black')
plt.title('Histograma da Feature 0')
plt.ylabel('Frequência')
plt.xlabel('Valor')
plt.show()

Valores nas extremidades (barras baixas) terão probabilidade baixa -> Score alto.
Valores no centro (barras altas) terão probabilidade alta -> Score baixo.

## Conclusão
Para auditoria em tempo real ou em datasets gigantescos (Big Data), HBOS é uma escolha excelente. A desvantagem é que ele não captura correlações entre features (ex: 'Idade alta' é OK, 'Salário baixo' é OK, mas 'Idade ALTA com Salário BAIXO' ele pode não pegar se olhar isoladamente, a menos que os valores individuais sejam extremos).